# **Tutorial 8: Paleoclimate Models**
**Week 1, Day 4, Paleoclimate**

**Content creators:** Sloane Garelick

**Content reviewers:** Brodie Pearson

**Content editors:** Rieke Schäfer, Agustina Pesce, Zahra Khodakaramimaghsoud

**Production editors:** TBD

**Our 2023 Sponsors:** TBD

###**Code and Data Sources**

Code for this tutorial is based on existing notebooks from LinkedEarth that explore [paleoclimate model-data comparisons](https://github.com/LinkedEarth/paleoHackathon/blob/main/notebooks/PaleoHack-nb08_Model-DataConfrontationInTimeDomain.ipynb) using [PMIP3 model simulations](https://github.com/LinkedEarth/paleoHackathon/blob/main/notebooks/PaleoHack-nb07_Model-DataConfrontationInFrequencyDomain.ipynb).

Data from the following sources are used in this tutorial:

*   Braconnot, P., Harrison, S., Kageyama, M. et al. Evaluation of climate models using palaeoclimatic data. Nature Clim Change 2, 417–424 (2012). https://doi.org/10.1038/nclimate1456
*   Tierney, J., Mayes, M., Meyer, N. et al. Late-twentieth-century warming in Lake Tanganyika unprecedented since AD 500. Nature Geosci 3, 422–425 (2010). https://doi.org/10.1038/ngeo865



















#**Tutorial 8 Objectives**

So far today, we've been focusing on how proxy-based reconstructions can be used to understand past variations in Earth's climate sytem. However, another useful tool in paleoclimate is the use of climate models.  

In this tutorial, you'll explore data from the [Paleoclimate Modelling Intercomparison Project 3 (PMIP3)](https://www.nature.com/articles/nclimate1456). More specifically, you'll be analyzing global mean surface temperature (GMST) data from  simulations for the past 1,000 years. 

You'll also compare the PMIP3 GMST data to a proxy-based reconstruction of temperature from Lake Tanganyika in East Africa [(Tierney et al., 2010)](https://www.nature.com/articles/ngeo865). Through this proxy-model comparison, you'll be able to assess the differences and similarities between the two datasets. 


By the end of this tutorial you will be able to:

*   Plot time series of paleoclimate model simulations
*   Compare and interpret proxy-based reconstructions of climate to paleoclimate model simulations of climate




In [ ]:
# !pip install cartopy
# !pip install pyleoclim
# !pip install pandas
# !pip install numpy
# !pip install xarray

In [ ]:
import pandas as pd
import numpy as np

import cartopy
import pyleoclim as pyleo

import xarray as xr


##Load proxy-based temperature reconstructions

The proxy record we'll be analyzing in this tutorial is a 1,000 year-long lake surface temperature reconstruction from [Tierney et al., 2010](https://https://doi.org/10.1038/ngeo865). This record is from Lake Taganyika in equatorial East Africa and is based on the TEX86 ratio, which is a temperature proxy derived from the distribution of the isoprenoid glycerol dialkyl glycerol tetraether (iGDGT) of archaeal membrane lipids. The organisms producing these iGDGT compounds alter the structure of the compound in response to changes in temperature, so by measure changes in the ratio of the different compounds, we can infer past changes in temperature.   

Let's start by loading the proxy data, saving it as a `Series` in Pyleoclim, and plotting a time series. 


In [ ]:
# fname = 'tang_sst.csv'
import pooch

data_path = pooch.retrieve(
    "https://osf.io/p8tx3/download",
    known_hash=None
)

proxy_temp = pd.read_csv(data_path)

In [ ]:
proxy_temp.head()

In [ ]:
proxy_t = pyleo.Series(
    time=proxy_temp['Year'],
    value= proxy_temp['LST'],
    time_name='Age',
    time_unit='years AD',
    value_name='Surface Temperature',
    value_unit='ºC',
    label='Lake Tanganyika Surface Temperature'
)

proxy_t.plot(color='C1')

Let's make some initial observations about the data:


*   What is the overall temperature trend over the past 1,000 years?
*   Where are the major increases or decrease in temperature? What could be the cause of these shifts in surface temperature? 



##Last Millenium PMIP3 GMST Data

We can now load GMST data from the PMIP3 simulations for the past 1,000 years [(Braconnot et al. 2012 )](https://https://doi.org/10.1038/nclimate1456). 


In [ ]:
# load the raw data
fname = 'PMIP3_GMST.txt'
data_path = pooch.retrieve(
    "https://osf.io/gw2m5/download",
    known_hash=None
)
df = pd.read_table(data_path)

# display the raw data
df

Note that the data file includes several ensemble members for Community Earth System Model (CESM) and Goddard Institute for Space Studies (GISS) simulations. Ensembles are essentially groups of climate model simulations used for climate projections, or in this case, reconstructions. You will learn about this in much more detail on W2D1: Climate Modeling.

For now, we can replace these with their ensemble mean series.


In [ ]:
# Create a new pandas.DataFrame to store the processed data
df_new = df.copy()

# Remove the data columns for CESM and GISS ensemble members
for i in range(10):
    df_new = df_new.drop([f'CESM_member_{i+1}'], axis=1)

df_new = df_new.drop(['GISS-E2-R_r1i1p127.1'], axis=1)
df_new = df_new.drop(['GISS-E2-R_r1i1p127'], axis=1)
df_new = df_new.drop(['GISS-E2-R_r1i1p121'], axis=1)

# calculate the ensemble mean for CESM and GISS, and add the results into the table
df_new['CESM'] = df[[
    'CESM_member_1',
    'CESM_member_2',
    'CESM_member_3',
    'CESM_member_4',
    'CESM_member_5',
    'CESM_member_6',
    'CESM_member_7',
    'CESM_member_8',
    'CESM_member_9',
    'CESM_member_10',
]].mean(axis=1)

df_new['GISS'] = df[[
    'GISS-E2-R_r1i1p127.1',
    'GISS-E2-R_r1i1p127',
    'GISS-E2-R_r1i1p121',
]].mean(axis=1)

In [ ]:
# display the processed data
df_new

In our new dataframe, you can now see that the ensemble members for CESM and GISS are now replaced with one ensemble mean for each model simulation.



Now we can create a Pyleoclim `Series` object for each simulated GMST time series, which will allow us to easily plot the time series for each simulation and perform data analysis using various built-in tools.

In [ ]:
# store each pyleoclim.Series() object into a dictionary
ts_dict = {}
for name in df_new.columns[1:]:
    ts_dict[name] = pyleo.Series(
        time=df_new['Year'].values,  # the time axis
        value=df_new[name].values,   # the value axis
        label=name,                  # optional metadata: the nickname of the series
        time_name='Time',            # optional metadata: the name of the time axis
        time_unit='yrs',             # optional metadata: the unit of the time axis
        value_name='GMST anom.',     # optional metadata: the name of the value axis
        value_unit='K',              # optional metadata: the unit of the value axis
    )

We can now plot each simulation. For example, let's plot the CCSM4 GMST:

In [ ]:
fig, ax = ts_dict['CCSM4'].plot()

You can try plotting GMST data from different simulations by calling a different model name in the cell abov, e.g., `fig, ax = ts_dict['HadCM3'].plot()`.

But what if we wanted to plot all of the PMIP3 time series at once? We can do that using the `MultipleSeries` object in Pyleoclim, which takes a list of `Series` objects as input. To do so, we have to convert the dictionary of `Series` into a list and then create a `MultipleSeries` object.

In [ ]:
ts_list = [v for k, v in ts_dict.items()]  # a pythonic way to convert the pyleo.Series items in the dictionary to a list
ms_pmip = pyleo.MultipleSeries(ts_list)

We can now plot all PMIP3 simulation time series at once:

In [ ]:
fig, ax = ms_pmip.plot(
    lgd_kwargs={
        'bbox_to_anchor': (1.25, 1),  # move the legend to the right side
    }
)

The GMST reconstructions from all of the PMIP3 simulations follow the same overall trend of relatively stable, long-term temperature from 800-1800 AD, followed by an increase in temperature over the past 200 years. What do you think is driving this recent warming trend?

Despite the long-term similarities, there are also noticeable differences between the GMST time series from each simulation.  

*   What do you observe about the various time series?
*   How are the GMST reconstructions from each simulation different?
*   What could be causing these differences?
*   How do we know which simulation is the most accurate and reliable?



One approach to trying to constrain climate model simulations as assess their ability to simulate past climate is to compare model data to comparable proxy data.

##Proxy-Model Comparisons

Proxy-based reconstructions of climate variables in the past can provide direct measurements of temperature, precipitation, greenhouse gas concentration, etc. Comparing proxy paleoclimate records with paleoclimate model simulations can help to clarify the interpretation of the proxy record and also help to improve the ability of climate models to simulate past variations in climate.

Here, we'll compare the proxy-based Lake Tanganyika surface temperature reconstruction we downloaded and plotted at the beginning of the tutorial, with the GMST PMIP3 simulations.

In [ ]:
#Standardize the proxy data
proxy_stnd = proxy_t.standardize()

In [ ]:
fig, ax = ms_pmip.plot(
    lgd_kwargs={
        'bbox_to_anchor': (1.25, 1),  # move the legend to the right side
    }
)

ax.set_ylabel('GMST anom.')
ax1 = ax.twinx()  # instantiate a second axes that shares the same x-axis
ax1.set_ylabel('Tanganyika LST')  # we already handled the x-label with ax1

proxy_stnd.plot(ax=ax1,color='black')
ax.set_xlim(xmin=850,xmax=2000)
ax.set_ylim(ymin=-4,ymax=2)
ax1.set_ylim(ymin=-4,ymax=2)

How do the model simulated GMST and proxy-based surface temperature compare?


*   Are the long term trends over the last millenium similar or different?
*   Is there more variability in the proxy or model temperatures? What might be causing this? 
*   How does the timing of temperature change compare?

This is just one simplified example of a proxy-model comparison. Larger-scale proxy-model comparisons can help to identify spatial patterns of temperature change and assess forcing mechanisms of paleoclimate variations. In W2D1, you'll spend more time exploring climate models.

In the next and final tutorial for today, you'll explore another tool for assessing past climate variability using both proxies and models.

In [ ]:
# Rieke Schäfer comments
# ======================

# Installations run well (maybe remove the comments for packages not needed?)
# for importing the path needs to be adapted, because it is set absolute not relative (but I have no idea in which context the particpants will get it, so might be fine)
# there seems to be some text missing under the objectives
# from the "model simulations" on there are many helpful comments in the code :)
# some text explaining the objective last plot would be nice
# the last cell does a spurious printing of the last setting ("(-4.0, 2.0)" in this case). Maybe someone with more experience with pyleoclim can figure out how to avoid that, I couldn't find it.